## GPU info

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri May 17 06:31:29 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   35C    P8              11W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Check if GPU is available
from tensorflow.python.client import device_lib

def get_gpu_details():
    devices = device_lib.list_local_devices()
    for device in devices:
        if device.device_type == 'GPU':
            print(f"Device Name: {device.name}")
            print(f"Memory Limit: {device.memory_limit} bytes")
            print(f"Description: {device.physical_device_desc}")

get_gpu_details()


Device Name: /device:GPU:0
Memory Limit: 21991653376 bytes
Description: device: 0, name: NVIDIA L4, pci bus id: 0000:00:03.0, compute capability: 8.9


In [4]:
# !pip install spacy
# !python -m spacy download en_core_web_sm

In [5]:
# !pip install pyarrow

In [6]:
# !pip install fastparquet

In [71]:
import os

import re
import string
import unicodedata
import nltk
# import spacy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Bidirectional, Concatenate, Layer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import tensorflow as tf

from tensorflow.keras.models import load_model

import pickle

In [72]:
# Download necessary NLTK resources
nltk.download('punkt')  # Tokenizer
nltk.download('wordnet')  # Lemmatizer
nltk.download('stopwords')  # Stopwords
nltk.download('omw-1.4') # Ensures multilingual contexts

# Stopwords list
stop_words = set(stopwords.words('english'))

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

initial_preprocessing = True

# # Load spaCy's English NLP model
# nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## Create prepocessing functions for initial text and later response generation preprocessing

In [73]:
def normalize_text(text: str) -> str:
    # Normalize Unicode string to NFKD form, remove non-ASCII characters, and then decode it back to a UTF-8 string
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    # Convert to lowercase
    text = text.lower()
    # Add a space before any punctuation mark (., !, or ?)
    text = re.sub(r"([.!?])", r" \1", text)
    # Handle contractions correctly by not adding space before apostrophe
    text = re.sub(r"(\b\w+)'(d|s|t|ll|ve|re)", r"\1'\2", text)
    # Replace any sequence of characters that are not letters, keep basic punctuation
    text = re.sub(r"[^a-z.,'!? ]", ' ', text)
    # Replace any sequence of whitespace characters with a single space and remove leading and trailing whitespace
    text = re.sub(r"\s+", r" ", text).strip()
    return text

def remove_names(text: str) -> str:
    # Use spaCy to detect and remove names from the text
    doc = nlp(text)
    filtered_text = ' '.join([token.text for token in doc if token.ent_type_ != 'PERSON']) # Takes really long time, exlude from chatbot input preprocessing
    return filtered_text

def preprocess_text(text: str) -> str:
    # Normalize text
    text = normalize_text(text)
    # Remove names using spaCy's NER
    if initial_preprocessing:
        text = remove_names(text)
    # # Remove punctuation
    # text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove stopwords and tokenize
    # words = word_tokenize(text) # More intelligent splitting
    # filtered_words = [word for word in words if word not in stop_words]
    # # Lemmatize words
    # lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]
    # Add <SOS> and <EOS> tokens, and join the list into a single string
    # return ' '.join(['sofs'] + lemmatized_words + ['eofs'])
    return 'sofs ' + text + ' eofs' # Chosen ['sofs', 'eofs'] because tokenizer removes everthing what is in <> or || and are not in dataset vocabulary

## Load the Tokenizer

In [74]:
# Load the tokenizer from file
data_dir = os.path.join(os.getcwd(), 'data')
tokenizer_path = os.path.join(data_dir, 'tokenizer.pickle')
with open(tokenizer_path, 'rb') as handle:
    tokenizer = pickle.load(handle)

In [75]:
print(tokenizer.word_index['sofs'], tokenizer.word_index['eofs']) # Checking if <start> and <end> tokens are in index (vocabulary)

1 2


In [76]:
# Top words in dictionary
from collections import OrderedDict

# Sort the word_counts dictionary by frequency in descending order
sorted_word_counts = OrderedDict(sorted(tokenizer.word_counts.items(), key=lambda x: x[1], reverse=True))

# Display the sorted word counts
print(list(sorted_word_counts.items())[:10])
print(list(sorted_word_counts.items())[-100:])

[('sofs', 304713), ('eofs', 304713), ('you', 148729), ('i', 142169), ('the', 99290), ('to', 80761), ('a', 71534), ("'s", 66252), ('it', 66206), ("n't", 55106)]
[('nihilistic', 1), ('freelancing', 1), ('gatherer', 1), ('overview', 1), ('retardant', 1), ('deploys', 1), ('beastie', 1), ('ozzfest', 1), ('russkie', 1), ('shavers', 1), ('mersh', 1), ('slovo', 1), ('dawning', 1), ('tshirt', 1), ('dishonorably', 1), ('vandals', 1), ('grozny', 1), ('lamborghini', 1), ('genoa', 1), ('pizda', 1), ('filament', 1), ('replicate', 1), ('solider', 1), ('secaucus', 1), ('athletics', 1), ('herded', 1), ('wolverine', 1), ('absorbs', 1), ('definitively', 1), ('poppycock', 1), ('rumous', 1), ('disinfectant', 1), ('celery', 1), ('cerebrum', 1), ('unashamedly', 1), ('dien', 1), ('gerhart', 1), ('mending', 1), ('galvanism', 1), ('equalize', 1), ('cerebrospinal', 1), ('madein', 1), ('froderick', 1), ('blindingly', 1), ('desserts', 1), ('impossibilities', 1), ('recesses', 1), ('ascend', 1), ('thunders', 1), ('k

## Load the Data

In [77]:
# Loading the DataFrame
file_path_parquet = os.path.join(data_dir, 'training_df_s2s.parquet')
training_data_final = pd.read_parquet(file_path_parquet)

training_data_final.head(10)

,ID_Input,Padded_Input_Sequences,ID_Response,Padded_Target_Sequences
0,L1044,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 37, 11, 6, 2]",L1045,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 37, 11, 31, 2]"
1,L984,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 51, 111, 2]",L985,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 4, 347, 46, 2]"
2,L924,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 897, 2]",L925,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 95, 8, 63, 2]"
3,L871,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 32, 2]",L872,"[0, 0, 1, 111, 3, 26, 118, 117, 129, 6, 650, 5..."
4,L870,"[1, 4, 24, 671, 3, 25, 55, 464, 3, 38, 711, 21...",L871,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 32, 2]"
5,L868,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 5, 224, 3, 2]",L869,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 40, 29, 885, 14, 9..."
6,L867,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 15, 74, 309, 2]",L868,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 5, 224, 3, 2]"
7,L866,"[0, 0, 0, 1, 4, 772, 3, 80, 44, 6, 5, 74, 309,...",L867,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 15, 74, 309, 2]"
8,L864,"[0, 0, 0, 0, 1, 17, 21, 6852, 2510, 4, 24, 40,...",L865,"[0, 1, 210, 197, 49, 4, 102, 6, 226, 57, 115, ..."
9,L863,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 15, 1075, 2]",L864,"[0, 0, 0, 0, 1, 17, 21, 6852, 2510, 4, 24, 40,..."


In [14]:
# len(training_data_final)

221616

In [15]:
# training_data_final = training_data_final.head(20000)

In [16]:
# len(training_data_final)

20000

# Encoder-decoder architecture with Attention Layer

In [78]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Bidirectional, Concatenate, Layer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import tensorflow as tf

## Attention Layer

In [79]:
class Attention(Layer):
    def __init__(self, units, **kwargs):
        super(Attention, self).__init__(**kwargs)
        self.units = units
        self.W1 = Dense(units)
        self.W2 = Dense(units)
        self.V = Dense(1)

    def call(self, query, values):
        query_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(query_with_time_axis)))
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

    def get_config(self):
        config = super().get_config()
        config.update({
            "units": self.units
        })
        return config

class AttentionLayer(Layer):
    def __init__(self, units, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
        self.units = units
        self.attention = Attention(units)

    def call(self, inputs):
        decoder_outputs, encoder_outputs = inputs
        batch_size = tf.shape(decoder_outputs)[0]
        sequence_length = tf.shape(decoder_outputs)[1]
        hidden_size = tf.shape(decoder_outputs)[2]

        # Tile encoder_outputs to match the batch and sequence length
        encoder_outputs = tf.tile(tf.expand_dims(encoder_outputs, 1), [1, sequence_length, 1, 1])
        encoder_outputs = tf.reshape(encoder_outputs, [batch_size * sequence_length, -1, hidden_size])

        # Flatten decoder_outputs to match encoder outputs
        decoder_outputs = tf.reshape(decoder_outputs, [batch_size * sequence_length, hidden_size])

        # Compute attention context vector
        context_vector, attention_weights = self.attention(decoder_outputs, encoder_outputs)
        context_vector = tf.reshape(context_vector, [batch_size, sequence_length, hidden_size])

        return context_vector

    def get_config(self):
        config = super().get_config()
        config.update({
            "units": self.units
        })
        return config


## Define the Model

In [80]:
input_sequences = np.array(training_data_final['Padded_Input_Sequences'].tolist())
target_sequences = np.array(training_data_final['Padded_Target_Sequences'].tolist())

# Splitting the data into training and validation sets
input_train, input_val, target_train, target_val = train_test_split(input_sequences, target_sequences, test_size=0.1, random_state=22)

# Building the model
vocab_size = len(tokenizer.word_index) + 1

# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(vocab_size, 50, mask_zero=True)(encoder_inputs)
encoder_lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional(
    LSTM(256, return_state=True, return_sequences=True))(encoder_embedding)
encoder_states = [Concatenate()([forward_h, backward_h]), Concatenate()([forward_c, backward_c])]
encoder_outputs = encoder_lstm

# Attention Mechanism
attention_units = 10
attention_layer = AttentionLayer(attention_units)

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(vocab_size, 50, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(512, return_sequences=True, return_state=True)
decoder_lstm_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

# Apply attention to each time step in the decoder
context_vectors = attention_layer([decoder_lstm_outputs, encoder_outputs])

decoder_concat_input = Concatenate(axis=-1)([context_vectors, decoder_lstm_outputs])
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_concat_input)

# Main Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_23 (InputLayer)       [(None, None)]               0         []                            
                                                                                                  
 embedding_7 (Embedding)     (None, None, 50)             2379000   ['input_23[0][0]']            
                                                                                                  
 input_24 (InputLayer)       [(None, None)]               0         []                            
                                                                                                  
 bidirectional_1 (Bidirecti  [(None, None, 512),          628736    ['embedding_7[0][0]']         
 onal)                        (None, 256),                                                 

## Load the Model

In [81]:
# from tensorflow.keras.models import load_model

# data_dir = os.path.join(os.getcwd(), 'data')
# file_path_h5 = os.path.join(data_dir, 's2s_model.h5')

# # Load the model
# custom_objects = {'Attention': Attention, 'AttentionLayer': AttentionLayer}
# model = load_model(file_path_h5, custom_objects=custom_objects)
# model.summary()

In [34]:
# from tensorflow.keras.models import load_model

# # Load the model
# model_path = 's2s_modelmodel_checkpoint_epoch_12.h5'
# custom_objects = {'Attention': Attention, 'AttentionLayer': AttentionLayer}
# model = load_model(model_path, custom_objects=custom_objects)

# # Summary of the loaded model
# model.summary()


## Train the Model

In [35]:
batch_size = 64
epochs = 50

# Prepare decoder input data that just contains the start token
decoder_input_train = np.hstack([np.zeros((target_train.shape[0], 1)), target_train[:, :-1]])
decoder_input_val = np.hstack([np.zeros((target_val.shape[0], 1)), target_val[:, :-1]])

# Ensure targets are expanded in dimension to match the output shape expected by sparse_categorical_crossentropy
target_train_exp = np.expand_dims(target_train, -1)
target_val_exp = np.expand_dims(target_val, -1)

# Checkpoint callback
# checkpoint_filepath = 'model_checkpoint_epoch_{epoch:02d}.h5'
checkpoint_filepath = '/content/drive/MyDrive/Colab Notebooks/models/model_checkpoint_epoch_{epoch:02d}.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    save_freq='epoch'
)

# Fit the model using the original integer labels
model.fit(
    [input_train, decoder_input_train], target_train_exp,
    validation_data=([input_val, decoder_input_val], target_val_exp),
    epochs=epochs, batch_size=batch_size, verbose=1,
    callbacks=[model_checkpoint_callback]
)

Epoch 1/50
282/282 [==============================] - 62s 192ms/step - loss: 2.7247 - accuracy: 0.3890 - val_loss: 5.6891 - val_accuracy: 0.1750
Epoch 2/50
282/282 [==============================] - 38s 134ms/step - loss: 2.6131 - accuracy: 0.4092 - val_loss: 5.7660 - val_accuracy: 0.1713
Epoch 3/50
282/282 [==============================] - 37s 131ms/step - loss: 2.4985 - accuracy: 0.4299 - val_loss: 5.8464 - val_accuracy: 0.1700
Epoch 4/50
282/282 [==============================] - 37s 130ms/step - loss: 2.3913 - accuracy: 0.4509 - val_loss: 5.9223 - val_accuracy: 0.1675
Epoch 5/50
282/282 [==============================] - 37s 130ms/step - loss: 2.2903 - accuracy: 0.4714 - val_loss: 5.9870 - val_accuracy: 0.1700
Epoch 6/50
282/282 [==============================] - 37s 131ms/step - loss: 2.1889 - accuracy: 0.4917 - val_loss: 6.0699 - val_accuracy: 0.1644
Epoch 7/50
282/282 [==============================] - 36s 129ms/step - loss: 2.0880 - accuracy: 0.5137 - val_loss: 6.1459 - val_ac

In [88]:
from tensorflow.keras.models import load_model

# Load the model
model_path = '/content/drive/MyDrive/Colab Notebooks/models/model_checkpoint_epoch_50.h5'
custom_objects = {'Attention': Attention, 'AttentionLayer': AttentionLayer}
model = load_model(model_path, custom_objects=custom_objects)

# Summary of the loaded model
model.summary()


Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_11 (InputLayer)       [(None, None)]               0         []                            
                                                                                                  
 embedding_4 (Embedding)     (None, None, 50)             2379000   ['input_11[0][0]']            
                                                                                                  
 input_12 (InputLayer)       [(None, None)]               0         []                            
                                                                                                  
 bidirectional_1 (Bidirecti  [(None, None, 512),          628736    ['embedding_4[0][0]']         
 onal)                        (None, 256),                                                  

In [89]:
# Encoder inference model
encoder_model = Model(encoder_inputs, [encoder_outputs, forward_h, forward_c, backward_h, backward_c])

# Decoder inference model
decoder_state_input_h = Input(shape=(512,))
decoder_state_input_c = Input(shape=(512,))
encoder_outputs_input = Input(shape=(None, 512))
decoder_inputs_single = Input(shape=(1,))

# Embedding layer
decoder_embedding_layer = Embedding(vocab_size, 50, mask_zero=True)
decoder_embedding_single = decoder_embedding_layer(decoder_inputs_single)

# LSTM layer
decoder_lstm_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding_single, initial_state=[decoder_state_input_h, decoder_state_input_c])

# Context vector from attention mechanism
context_vector = attention_layer([decoder_lstm_outputs, encoder_outputs_input])

# Concatenate context vector with LSTM outputs
decoder_concat_input = Concatenate(axis=-1)([context_vector, decoder_lstm_outputs])

# Dense layer
decoder_outputs = decoder_dense(decoder_concat_input)

# Decoder inference model
decoder_model = Model(
    [decoder_inputs_single, encoder_outputs_input, decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs, state_h, state_c]
)


## Save the model

In [90]:
# data_dir = os.path.join(os.getcwd(), 'data')
# file_path_h5 = os.path.join(data_dir, 's2s_model.h5')
# model.save(file_path_h5)

## Generate responses

In [94]:
initial_preprocessing = False # Excepts spaCy to detect and remove names from the text
max_length = 15

# Generate response function
def generate_response(input_text: str) -> str:
    text = preprocess_text(input_text)
    print(f"Preprocessed text: {text}")
    input_seq = tokenizer.texts_to_sequences([text])
    print(f"Input sequence: {input_seq}")
    input_seq = pad_sequences(input_seq, maxlen=max_length, padding='post', truncating='post')
    print(f"Padded input sequence: {input_seq}")

    # Get the encoder states and encoder outputs
    encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_model.predict(input_seq)
    state_h = np.concatenate([forward_h, backward_h], axis=-1)
    state_c = np.concatenate([forward_c, backward_c], axis=-1)
    states_value = [state_h, state_c]

    # Prepare the target sequence with the start token
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tokenizer.word_index['start']

    stop_condition = False
    decoded_sentence = ''
    tokens_generated = 0

    while not stop_condition:
        decoder_outputs, h, c = decoder_model.predict([target_seq, encoder_outputs, state_h, state_c], verbose=0)

        sampled_token_index = np.argmax(decoder_outputs[0, -1, :])
        sampled_token = tokenizer.index_word.get(sampled_token_index, '')

        if sampled_token == 'end' or tokens_generated > max_length:
            stop_condition = True
        else:
            decoded_sentence += ' ' + sampled_token
            tokens_generated += 1

            target_seq = np.zeros((1, 1))
            target_seq[0, 0] = sampled_token_index
            states_value = [h, c]

    return decoded_sentence.strip()

## Testing

In [95]:
# Testing
print("\nUser:     Is she okay?")
print("Bot:          ", generate_response('Is she okay?'))
print("-----------------------------")
print("\nUser:     How are you feeling today?")
print("Bot:          ", generate_response('How are you feeling today?'))
print("-----------------------------")
print("\nUser:     Hi there!")
print("Bot:          ", generate_response('Hi there!'))
print("-----------------------------")
print("\nUser:     Can you tell me the weather forecast for today?")
print("Bot:          ", generate_response('Can you tell me the weather forecast for today?'))
print("-----------------------------")
print("\nUser:     I think artificial intelligence is changing the world.")
print("Bot:          ", generate_response('I think artificial intelligence is changing the world.'))
print("-----------------------------")
print("\nUser:     Any good movie recommendations?")
print("Bot:          ", generate_response('Any good movie recommendations?'))
print("-----------------------------")
print("\nUser:     What do you mean by that?")
print("Bot:          ", generate_response('What do you mean by that?'))
print("-----------------------------")
print("\nUser:     I'm feeling really sad today.")
print("Bot:          ", generate_response("I'm feeling really sad today."))
print("-----------------------------")
print("\nUser:     What are the implications of quantum computing on cybersecurity?")
print("Bot:          ", generate_response('What are the implications of quantum computing on cybersecurity?'))
print("-----------------------------")
print("\nUser:     Why did the chicken cross the road?")
print("Bot:          ", generate_response('Why did the chicken cross the road?'))
print("-----------------------------")
print("\nUser:     Can you explain the plot of The Matrix?")
print("Bot:          ", generate_response('Can you explain the plot of The Matrix?'))


User:     Is she okay?
Preprocessed text: sofs is she okay ? eofs
Input sequence: [[1, 18, 51, 111, 2]]
Padded input sequence: [[  1  18  51 111   2   0   0   0   0   0   0   0   0   0   0]]
1/1 [==============================] - 0s 29ms/step
Bot:           fahzer lcd samoans catholicism inspire overflow effective retina overflow effective retina overflow effective retina overflow effective
-----------------------------

User:     How are you feeling today?
Preprocessed text: sofs how are you feeling today ? eofs
Input sequence: [[1, 55, 34, 3, 549, 326, 2]]
Padded input sequence: [[  1  55  34   3 549 326   2   0   0   0   0   0   0   0   0]]
1/1 [==============================] - 0s 23ms/step
Bot:           conroy anouk eyefuck cynical sharply surround java sailors lookee apeshit cynical sharply surround java sailors lookee
-----------------------------

User:     Hi there!
Preprocessed text: sofs hi there ! eofs
Input sequence: [[1, 357, 42, 2]]
Padded input sequence: [[  1 357  42

## Save the model in Google Drive

In [93]:
# data_dir = '/content/drive/MyDrive/Colab Notebooks/models'
# if not os.path.exists(data_dir):
#     os.makedirs(data_dir)

# file_path_h5 = os.path.join(data_dir, 's2s_model.h5')
# model.save(file_path_h5)
